### Lightning factors DL/ML code into three types:

 * Research code

 * Engineering code

 * Non-essential code

### The Model

In [9]:
import torch
from torch.nn import functional as F
from torch import nn
import pytorch_lightning as pl
from torch.optim import Adam
from pytorch_lightning import Trainer



class LitMNIST(pl.LightningModule):

  def __init__(self):
    super().__init__()

    # mnist images are (1, 28, 28) (channels, width, height)
    self.layer_1 = torch.nn.Linear(28 * 28, 128)
    self.layer_2 = torch.nn.Linear(128, 256)
    self.layer_3 = torch.nn.Linear(256, 10)

  def forward(self, x):
    batch_size, channels, width, height = x.size()

    # (b, 1, 28, 28) -> (b, 1*28*28)
    x = x.view(batch_size, -1)

    # layer 1
    x = self.layer_1(x)
    x = torch.relu(x)

    # layer 2
    x = self.layer_2(x)
    x = torch.relu(x)

    # layer 3
    x = self.layer_3(x)

    # probability distribution over labels
    x = torch.log_softmax(x, dim=1)

    return x

Notice this is a LightningModule instead of a torch.nn.Module. A LightningModule is equivalent to a PyTorch Module except it has added functionality. However, you can use it EXACTLY the same as you would a PyTorch Module.

In [10]:
net = LitMNIST()
x = torch.Tensor(1, 1, 28, 28)
out = net(x)
out

tensor([[-2.3678e+34, -2.1916e+34,  0.0000e+00, -3.0966e+32, -9.2373e+33,
         -9.9944e+33, -6.1999e+33, -8.8007e+33, -2.2410e+34, -1.5735e+34]],
       grad_fn=<LogSoftmaxBackward>)

### Data
The Lightning Module organizes your dataloaders and data processing as well. Here’s the PyTorch code for loading MNIST
```python
class LitMNIST(pl.LightningModule):

  def prepare_data(self):
    # stuff here is done once at the very beginning of training
    # before any distributed training starts

    # download stuff
    # save to disk
    # etc...

  def train_dataloader(self):
    # data transforms
    # dataset creation
    # return a DataLoader
```



In [11]:
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import os
from torchvision import datasets, transforms


# transforms
# prepare transforms standard to MNIST
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))])

# data
mnist_train = MNIST(os.getcwd(), train=True, download=True)
mnist_train = DataLoader(mnist_train, batch_size=64)


class LitMNIST(pl.LightningModule):

  def prepare_data(self):
    # download only
    MNIST(os.getcwd(), train=True, download=True)

  def train_dataloader(self):
    # no download, just transform
    transform=transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))])
    mnist_train = MNIST(os.getcwd(), train=True, download=False,
                        transform=transform)
    return DataLoader(mnist_train, batch_size=64)

### Optimizer
In Lightning optimizers are under the configure_optimizers method.



In [12]:
class LitMNIST(pl.LightningModule):

  def configure_optimizers(self):
    return Adam(self.parameters(), lr=1e-3)

### Training Step

Training step in pytorch typically looks like below:
```python
for epoch in epochs:
    for batch in data:
        # TRAINING STEP
        # ....
        # TRAINING STEP
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
```
In Lightning, everything that is in the training step gets organized under the training_step function in the LightningModule



In [13]:
class LitMNIST(pl.LightningModule):

  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = F.nll_loss(logits, y)
    return {'loss': loss}
    # return loss (also works)

### Training - Combining all Togeter

So far we defined 4 key ingredients in pure PyTorch but organized the code inside the LightningModule.

* Model.

* Training data.

* Optimizer.

* What happens in the training loop.

For clarity, we’ll recall that the full LightningModule now looks like this.

In [14]:
class LitMNIST(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.layer_1 = torch.nn.Linear(28 * 28, 128)
    self.layer_2 = torch.nn.Linear(128, 256)
    self.layer_3 = torch.nn.Linear(256, 10)

  def forward(self, x):
    batch_size, channels, width, height = x.size()
    x = x.view(batch_size, -1)
    x = self.layer_1(x)
    x = torch.relu(x)
    x = self.layer_2(x)
    x = torch.relu(x)
    x = self.layer_3(x)
    x = torch.log_softmax(x, dim=1)
    return x

  def train_dataloader(self):
    transform=transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))])
    mnist_train = MNIST(os.getcwd(), train=True, download=False, transform=transform)
    return DataLoader(mnist_train, batch_size=64)

  def configure_optimizers(self):
    return Adam(self.parameters(), lr=1e-3)

  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = F.nll_loss(logits, y)

    # add logging
    logs = {'loss': loss}
    return {'loss': loss, 'log': logs}

### Logging

When we added the log key in the return dictionary it went into the built in tensorboard logger. But you could have also logged by calling:

```python
def training_step(self, batch, batch_idx):
    # ...
    loss = ...
    self.logger.summary.scalar('loss', loss)
```

### GPU Training


In [15]:
from pytorch_lightning import Trainer


model = LitMNIST()
trainer = Trainer(gpus=1,max_epochs=4)
trainer.fit(model)

INFO:lightning:GPU available: True, used: True
INFO:lightning:VISIBLE GPUS: 0
/home/av6101604/env/lib64/python3.6/site-packages/pytorch_lightning/loggers/tensorboard.py:106: UserWarning: Hyperparameter logging is not available for Torch version 1.2.0. Skipping log_hyperparams. Upgrade to Torch 1.3.0 or above to enable hyperparameter logging.
  f"Hyperparameter logging is not available for Torch version {torch.__version__}."
INFO:lightning:
  | Name    | Type   | Params
-------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33 K  
2 | layer_3 | Linear | 2 K   


/home/av6101604/env/lib64/python3.6/site-packages/pytorch_lightning/utilities/warnings.py:18: RuntimeWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  warnings.warn(*args, **kwargs)
/home/av6101604/env/lib64/python3.6/site-packages/pytorch_lightning/utilities/warnings.py:18: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1

### Start Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/